In [1]:
import ast
import os
import csv
import sys
import nltk
import bs4 as bs
import string

maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [2]:
def get_set(set_string):
    if set_string == 'set()':
        return set()
    else:
        return ast.literal_eval(set_string)
    
def rem_end(text, suffix):
    if text.endswith(suffix):
        return text[:-len(suffix)]
    return text

def is_title(name):
    if len(name) >= 3 and name[-3] == '(' and name[-2].isdigit() and name[-1] == ')':
        return True
    else:
        return False

def valid_entity(name):
    if is_title(name) and (len(name) == 3 or name[-4] in string.punctuation):
        return False
    return True

In [4]:
bad_ent = []
for section in range(1, 10):

    directory = 'D:\\KGP\\Extractions\\man' + str(section) + '\\'
    all_files = os.listdir(directory)

    n = 1
    for file_name in all_files:

        sys.stdout.flush()
        sys.stdout.write("Section : %d Progress: %d / %d / %d \r" % (section, n, len(all_files), len(bad_ent)))

        ent_set = set()
        with open(directory + file_name, 'r') as ent_file:
            csv_reader = csv.DictReader(ent_file, delimiter = '\t') 
            max_ent_len = 0
            for entity in csv_reader:
                name = entity['name']
                if valid_entity(name):
                    ent_set.add(name.replace(' ', ''))
                    max_ent_len = max([max_ent_len, len(nltk.word_tokenize(name))])

        txt_file_name = rem_end(file_name, 'txt') + 'html'
        with open('D:\\KGP\\man' + str(section) + '\\' + txt_file_name, 'rb') as sauce:
            soup = bs.BeautifulSoup(sauce, 'lxml')
            sentences = nltk.sent_tokenize(soup.get_text())
            for sentence in sentences:
                words = nltk.word_tokenize(sentence)
                for length in range(1, max_ent_len + 1):
                    for start in range(len(words) - length + 1):
                        candidate = ''.join(words[start: start + length])
                        ent_set.discard(candidate)
                        if is_title(candidate):
                            ent_set.discard(candidate[:-3])
                        else:
                            ent_set.discard(candidate + '(' + str(section) + ')')

        for entity in ent_set:
            bad_ent.append(str(section) + ' ' + file_name + ' ' + entity)

        n += 1

KeyboardInterrupt: 

In [ ]:
bad_ent

In [ ]:
nltk.sent_tokenize('Ram is bad. Sita is good.')

In [ ]:
nltk.word_tokenize('Ram is bad.')